In [0]:
%python

value = 'Spark'
print("This! is%s Notebook!" % value)

This! isSpark Notebook!

In [0]:
# AWS Access key 가져오기
ACCESS_KEY_ID = "???????"
SECRET_ACCESS_KEY = "??????"
ENCODED_SECRET_ACCESS_KEY = SECRET_ACCESS_KEY.replace("/", "%2F")

# 실습자료는 Amazon Web Service(AWS)의 S3 bucket s3a://edwith-pyspark-dataset에 저장되어 있음
# AWS의 S3 bucket에 연결.
dbutils.fs.mount("s3a://%s:%s@edwith-pyspark-dataset" % (ACCESS_KEY_ID, ENCODED_SECRET_ACCESS_KEY), "/mnt/us-carrier-dataset")

# dbutils.fs.unmount("/mnt/us-carrier-dataset") # mount 해제


Out[6]: True

In [0]:
# 이제 /mnt/us-carrier-dataset 위치에서 자료 읽기 가능.
# 파일 리스트 확인시 ls 명령어를 사용해야 하지만, databricks cloud 환경에서는 display 함수를 이용해 확인 가능.
display(dbutils.fs.ls("/mnt/us-carrier-dataset"))

path,name,size
dbfs:/mnt/us-carrier-dataset/1987.csv,1987.csv,127162942
dbfs:/mnt/us-carrier-dataset/1988.csv,1988.csv,501039472
dbfs:/mnt/us-carrier-dataset/1989.csv,1989.csv,486518821
dbfs:/mnt/us-carrier-dataset/1990.csv,1990.csv,509194687
dbfs:/mnt/us-carrier-dataset/1991.csv,1991.csv,491210093
dbfs:/mnt/us-carrier-dataset/1992.csv,1992.csv,492313731
dbfs:/mnt/us-carrier-dataset/1993.csv,1993.csv,490753652
dbfs:/mnt/us-carrier-dataset/1994.csv,1994.csv,501558665
dbfs:/mnt/us-carrier-dataset/1995.csv,1995.csv,530751568
dbfs:/mnt/us-carrier-dataset/1996.csv,1996.csv,533922363


In [0]:
# SparkSession 객체 생성 후, 데이터 Load, 데이터 확인하기

# SparkSession 객체 생성
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Spark EDA example on us aircraft data ") \
  .getOrCreate()

In [0]:
# 데이터 읽기
raw_df = spark.read.csv("/mnt/us-carrier-dataset", header = 'True', inferSchema='True')

In [0]:
# 스키마 형태 표시
raw_df.printSchema()

root
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- DayofMonth: integer (nullable = true)
-- DayOfWeek: integer (nullable = true)
-- DepTime: string (nullable = true)
-- CRSDepTime: integer (nullable = true)
-- ArrTime: string (nullable = true)
-- CRSArrTime: integer (nullable = true)
-- UniqueCarrier: string (nullable = true)
-- FlightNum: integer (nullable = true)
-- TailNum: string (nullable = true)
-- ActualElapsedTime: string (nullable = true)
-- CRSElapsedTime: string (nullable = true)
-- AirTime: string (nullable = true)
-- ArrDelay: string (nullable = true)
-- DepDelay: string (nullable = true)
-- Origin: string (nullable = true)
-- Dest: string (nullable = true)
-- Distance: string (nullable = true)
-- TaxiIn: string (nullable = true)
-- TaxiOut: string (nullable = true)
-- Cancelled: integer (nullable = true)
-- CancellationCode: string (nullable = true)
-- Diverted: integer (nullable = true)
-- CarrierDelay: string (nullable = true)
-- WeatherDelay: string (nullable = true)
-- NASDelay: string (nullable = true)
-- SecurityDelay: string (nullable = true)
-- LateAircraftDelay: string (nullable = true)

In [0]:
# 데이터 개수 표시
# 대략 22년간의 데이터, 1억2천만개 정도
raw_df.count()

Out[11]: 123534969

In [0]:
# 데이터 확인
raw_df.show(30) # 30개정도만 표시

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
1988| 1| 9| 6| 1348| 1331| 1458| 1435| PI| 942| NA| 70| 64| NA| 23| 17| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 10| 7| 1334| 1331| 1443| 1435| PI| 942| NA| 69| 64| NA| 8| 3| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 11| 1| 1446| 1331| 1553| 1435| PI| 942| NA| 67| 64| NA| 78| 75| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 12| 2| 1334| 1331| 1438| 1435| PI| 942| NA| 64| 64| NA| 3| 3| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 13| 3| 1341| 1331| 1503| 1435| PI| 942| NA| 82| 64| NA| 28| 10| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 14| 4| 1332| 1331| 1447| 1435| PI| 942| NA| 75| 64| NA| 12| 1| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 15| 5| 1331| 1331| 1434| 1435| PI| 942| NA| 63| 64| NA| -1| 0| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 16| 6| 1327| 1331| 1427| 1435| PI| 942| NA| 60| 64| NA| -8| -4| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 17| 7| 1331| 1331| 1440| 1435| PI| 942| NA| 69| 64| NA| 5| 0| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 18| 1| 1349| 1331| 1519| 1435| PI| 942| NA| 90| 64| NA| 44| 18| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 19| 2| 1337| 1331| 1438| 1435| PI| 942| NA| 61| 64| NA| 3| 6| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 20| 3| 1337| 1331| 1452| 1435| PI| 942| NA| 75| 64| NA| 17| 6| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 21| 4| 1330| 1331| 1459| 1435| PI| 942| NA| 89| 64| NA| 24| -1| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 22| 5| 1331| 1331| 1434| 1435| PI| 942| NA| 63| 64| NA| -1| 0| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 23| 6| 1333| 1331| 1436| 1435| PI| 942| NA| 63| 64| NA| 1| 2| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 24| 7| 1328| 1331| 1430| 1435| PI| 942| NA| 62| 64| NA| -5| -3| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 25| 1| 1330| 1331| 1444| 1435| PI| 942| NA| 74| 64| NA| 9| -1| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 26| 2| 1407| 1331| 1518| 1435| PI| 942| NA| 71| 64| NA| 43| 36| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 27| 3| 1332| 1331| 1445| 1435| PI| 942| NA| 73| 64| NA| 10| 1| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 28| 4| 1331| 1331| 1438| 1435| PI| 942| NA| 67| 64| NA| 3| 0| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 29| 5| 1339| 1331| 1442| 1435| PI| 942| NA| 63| 64| NA| 7| 8| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 30| 6| 1332| 1331| 1435| 1435| PI| 942| NA| 63| 64| NA| 0| 1| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 31| 7| 1329| 1331| 1434| 1435| PI| 942| NA| 65| 64| NA| -1| -2| SYR| BWI| 273| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 1| 5| 947| 950| 1050| 1050| PI| 943| NA| 63| 60| NA| 0| -3| LGA| SYR| 198| NA| NA| 0| NA| 0| NA| NA| NA| NA| NA|
1988| 1| 3| 7| 957| 950| 1048| 1050| PI| 943| NA| 51| 60| NA| -2| 7| LGA| SYR| 198| NA| NA| 0| NA|

In [0]:
# 정돈된 테이블 형태로 데이터 확인.
# databricks cloud상에서 gui형태로 그래프 그리기같은 기능 제공.
display(raw_df)

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1988,1,9,6,1348,1331,1458,1435,PI,942,NA,70,64,NA,23,17,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,10,7,1334,1331,1443,1435,PI,942,NA,69,64,NA,8,3,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,11,1,1446,1331,1553,1435,PI,942,NA,67,64,NA,78,75,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,12,2,1334,1331,1438,1435,PI,942,NA,64,64,NA,3,3,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,13,3,1341,1331,1503,1435,PI,942,NA,82,64,NA,28,10,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,14,4,1332,1331,1447,1435,PI,942,NA,75,64,NA,12,1,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,15,5,1331,1331,1434,1435,PI,942,NA,63,64,NA,-1,0,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,16,6,1327,1331,1427,1435,PI,942,NA,60,64,NA,-8,-4,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,17,7,1331,1331,1440,1435,PI,942,NA,69,64,NA,5,0,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,18,1,1349,1331,1519,1435,PI,942,NA,90,64,NA,44,18,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA


In [0]:
# column 값의 분포 상태를 나타내는 dataframe 
raw_df.describe()

Out[14]: DataFrame[summary: string, Year: string, Month: string, DayofMonth: string, DayOfWeek: string, DepTime: string, CRSDepTime: string, ArrTime: string, CRSArrTime: string, UniqueCarrier: string, FlightNum: string, TailNum: string, ActualElapsedTime: string, CRSElapsedTime: string, AirTime: string, ArrDelay: string, DepDelay: string, Origin: string, Dest: string, Distance: string, TaxiIn: string, TaxiOut: string, Cancelled: string, CancellationCode: string, Diverted: string, CarrierDelay: string, WeatherDelay: string, NASDelay: string, SecurityDelay: string, LateAircraftDelay: string]

In [0]:
# 데이터 전처리
# 필요 함수 import
from pyspark.sql.functions import udf # 사용자 정의 함수
from pyspark.sql.types import BooleanType, IntegerType, StringType # 데이터 타입

In [0]:
# NA로 표시되어 있는 NULL값을 변경.
def cleanString(value):
  if (value == None) or (value == 'NA'):
    return None
  else:
    value
    
# string으로 설정된 숫자값을 integer로 변경.
def stringToInteger(value):
  if value == 'NA':
    return None
  else:
    return int(value)
  
# integer 값을 boolean형으로 변경.
def integerToBoolean(value):
  return False if value == 0 else True

In [0]:
# 사용자 정의 함수(UDF) 설정.
# Spark SQL함수를 사용자가 정의할 수 있도록 해줌.
cleanStrFunction = udf(stringToInteger, IntegerType())
cleanIntegerFunction = udf(integerToBoolean, BooleanType())
cleanStringFunction = udf(cleanString, StringType())

In [0]:
# UDF 이용해 데이터 전처리하기 

us_carrier_df = raw_df \
    .drop(
      # 사용하지 않을 column 삭제
      'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'AirTime', 'ArrDelay', 'DepDelay', 'TaxiIn', 'TaxiOut', 'CancellationCode', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay') \
    .withColumn(
      # 'NA' 항목 null로 변경하고 Integer 형으로 변경
      'ActualElapsedTime', cleanStrFunction('ActualElapsedTime')) \
    .withColumn(
      # 'NA' 항목 null로 변경하고 Integer 형으로 변경
      'CRSElapsedTime', cleanStrFunction('CRSElapsedTime')) \
    .withColumn(
      # 'NA' 항목 null로 변경하고 Integer 형으로 변경
      'Distance', cleanStrFunction('Distance')) \
    .withColumn(
      # 'NA' 항목 null로 변경하고 Boolean 형으로 변경
      'Cancelled', cleanIntegerFunction('Cancelled')) \
    .withColumn(
      # 'NA' 항목 null로 변경하고 Boolean 형으로 변경
      'Diverted', cleanIntegerFunction('Diverted'))

In [0]:
us_carrier_df.printSchema()

root
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- DayofMonth: integer (nullable = true)
-- DayOfWeek: integer (nullable = true)
-- UniqueCarrier: string (nullable = true)
-- FlightNum: integer (nullable = true)
-- TailNum: string (nullable = true)
-- ActualElapsedTime: integer (nullable = true)
-- CRSElapsedTime: integer (nullable = true)
-- Origin: string (nullable = true)
-- Dest: string (nullable = true)
-- Distance: integer (nullable = true)
-- Cancelled: boolean (nullable = true)
-- Diverted: boolean (nullable = true)

In [0]:
us_carrier_df.show()

+----+-----+----------+---------+-------------+---------+-------+-----------------+--------------+------+----+--------+---------+--------+
Year|Month|DayofMonth|DayOfWeek|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|Origin|Dest|Distance|Cancelled|Diverted|
+----+-----+----------+---------+-------------+---------+-------+-----------------+--------------+------+----+--------+---------+--------+
1988| 1| 9| 6| PI| 942| NA| 70| 64| SYR| BWI| 273| false| false|
1988| 1| 10| 7| PI| 942| NA| 69| 64| SYR| BWI| 273| false| false|
1988| 1| 11| 1| PI| 942| NA| 67| 64| SYR| BWI| 273| false| false|
1988| 1| 12| 2| PI| 942| NA| 64| 64| SYR| BWI| 273| false| false|
1988| 1| 13| 3| PI| 942| NA| 82| 64| SYR| BWI| 273| false| false|
1988| 1| 14| 4| PI| 942| NA| 75| 64| SYR| BWI| 273| false| false|
1988| 1| 15| 5| PI| 942| NA| 63| 64| SYR| BWI| 273| false| false|
1988| 1| 16| 6| PI| 942| NA| 60| 64| SYR| BWI| 273| false| false|
1988| 1| 17| 7| PI| 942| NA| 69| 64| SYR| BWI| 273| false| false|
1988| 1| 18| 1| PI| 942| NA| 90| 64| SYR| BWI| 273| false| false|
1988| 1| 19| 2| PI| 942| NA| 61| 64| SYR| BWI| 273| false| false|
1988| 1| 20| 3| PI| 942| NA| 75| 64| SYR| BWI| 273| false| false|
1988| 1| 21| 4| PI| 942| NA| 89| 64| SYR| BWI| 273| false| false|
1988| 1| 22| 5| PI| 942| NA| 63| 64| SYR| BWI| 273| false| false|
1988| 1| 23| 6| PI| 942| NA| 63| 64| SYR| BWI| 273| false| false|
1988| 1| 24| 7| PI| 942| NA| 62| 64| SYR| BWI| 273| false| false|
1988| 1| 25| 1| PI| 942| NA| 74| 64| SYR| BWI| 273| false| false|
1988| 1| 26| 2| PI| 942| NA| 71| 64| SYR| BWI| 273| false| false|
1988| 1| 27| 3| PI| 942| NA| 73| 64| SYR| BWI| 273| false| false|
1988| 1| 28| 4| PI| 942| NA| 67| 64| SYR| BWI| 273| false| false|
+----+-----+----------+---------+-------------+---------+-------+-----------------+--------------+------+----+--------+---------+--------+
only showing top 20 rows

In [0]:
# cleanStringFunction(us_carrier_df)
us_carrier_df = us_carrier_df.replace('NA', None) # 처리하지못한 NA값

In [0]:
display(us_carrier_df)

Year,Month,DayofMonth,DayOfWeek,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,Origin,Dest,Distance,Cancelled,Diverted
1988,1,9,6,PI,942,null,70,64,SYR,BWI,273,false,false
1988,1,10,7,PI,942,null,69,64,SYR,BWI,273,false,false
1988,1,11,1,PI,942,null,67,64,SYR,BWI,273,false,false
1988,1,12,2,PI,942,null,64,64,SYR,BWI,273,false,false
1988,1,13,3,PI,942,null,82,64,SYR,BWI,273,false,false
1988,1,14,4,PI,942,null,75,64,SYR,BWI,273,false,false
1988,1,15,5,PI,942,null,63,64,SYR,BWI,273,false,false
1988,1,16,6,PI,942,null,60,64,SYR,BWI,273,false,false
1988,1,17,7,PI,942,null,69,64,SYR,BWI,273,false,false
1988,1,18,1,PI,942,null,90,64,SYR,BWI,273,false,false


In [0]:
# 추후 사용할 자료는 클러스터 메모리상에 남겨두고 삭제하지 않도록 설정.
us_carrier_df.cache()

Out[29]: DataFrame[Year: int, Month: int, DayofMonth: int, DayOfWeek: int, UniqueCarrier: string, FlightNum: int, TailNum: string, ActualElapsedTime: int, CRSElapsedTime: int, Origin: string, Dest: string, Distance: int, Cancelled: boolean, Diverted: boolean]

In [0]:
# SQL 질의 사용을 위해 DataFrame을 전역 임시 뷰로 저장.
us_carrier_df.createGlobalTempView("us_carrier")